In [2]:
import os
import pandas as pd
from bbox import * 
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
#for image inpainting
from skimage import data
from skimage.morphology import disk, binary_dilation
from skimage.restoration import inpaint
import cv2 as cv
import xesmf as xe

In [3]:
##################################
# Create DataFrames
##################################

files = os.listdir('../../data/tempo_data/')
files = [file for file in files if file[:23]=='satellite_null_data_raw']

df = pd.DataFrame()
for file in files:
    temp = pd.read_csv('../../data/tempo_data/'+file)
    df = pd.concat([df, temp])
    
full_size = 240*530
df['percent_null'] =round((df['null_count']/full_size)*100, 2)    
df = df[df['percent_null']<100]
df['hour'] = pd.to_datetime(df['time']).dt.floor('h')
df=df.drop_duplicates()

df = df[df['percent_null']<=5]
df=df.sort_values(by=['hour','null_count', 'time'])
valid_hours = df.groupby('hour').first().reset_index()
valid_hours.to_csv('../../data/tempo_data/valid_no2_hours.csv')

In [4]:
def inpainting(data):
    """
    Fills in missing values in the input data using inpainting techniques.
    
    Parameters:
    data (numpy.ndarray): Input array with potential NaN values.

    Returns:
    numpy.ndarray: Array with NaN values filled using inpainting, 
                   or the original array if no NaNs are present.
    """
    # Mask for NaN values
    data_copy = np.copy(data)
    na_data_mask = np.isnan(data_copy)

    # If no NaN values, return data as is
    if not na_data_mask.any():
        return data

    # Replace NaNs with the initial median value
    initial_value = np.nanmedian(data_copy)
    data_copy = data_copy.astype('float32')
    data_copy[na_data_mask] = initial_value

    # Inpainting to fill missing data
    filled_data = cv.inpaint(data_copy, na_data_mask.astype('uint8'), 10, cv.INPAINT_NS)
    return filled_data

In [5]:
timestamps_dt = pd.to_datetime(valid_hours['time'])
unique_months = timestamps_dt.dt.to_period('M').unique()

In [6]:
for month in unique_months[21:]:
    month_str = str(month)
    monthly_xr_list = []

    month_timestamps = timestamps_dt[timestamps_dt.dt.to_period('M') == month]
    days =month_timestamps.dt.to_period('d').unique()

    for day in days:
        day_str = str(day)
        day_timestamps = month_timestamps[month_timestamps.dt.to_period('d') == day]
        
        ds = xr.open_dataset(f'../../data/tempo_data/no2_daily_files/tempo_{day_str}.nc', engine='netcdf4')

        ds['time'] = np.array(ds['time'], dtype='datetime64[ns]')
        day_timestamps = np.array(day_timestamps, dtype='datetime64[ns]')
        ds = ds.sel(time=~ds.indexes['time'].duplicated())
        #ADDED LINE BELOW WITHOUT CHECKING
        day_timestamps=np.array(list(set(day_timestamps).intersection(set(np.array(ds['time'])))))
        ds = ds.sel(time=day_timestamps)

        lat_new = np.arange(lat_min+0.005, lat_max, 0.01)
        lon_new = np.arange(lon_min+0.005, lon_max, 0.01)

        # Divide by 1000000000000000
        ds['vertical_column_troposphere'] = ds['vertical_column_troposphere']/1000000000000000

        #fill nas
        for var in list(ds.data_vars.keys()):

            if var == 'main_data_quality_flag':
                ds['main_data_quality_flag']=ds['main_data_quality_flag'].fillna(2)
            else:
                inpainted_results = np.empty_like(ds[f"{var}"])

                for i in range(len(ds.time)):
                    data = ds.isel(time=i)[f"{var}"]
                    data_arr = np.array(data)
                    filled = inpainting(data_arr)

                    inpainted_results[i, :, :] = filled

                ds[f"{var}"] = (('time', 'latitude', 'longitude'), inpainted_results)

        ds_out = xr.Dataset(
            {
                "latitude": (["latitude"], lat_new),
                "longitude": (["longitude"], lon_new),
            }
        )

        # Create the regridder object.
        regridder = xe.Regridder(ds, ds_out, "bilinear")

        # Apply the regridding operation.
        ds = regridder(ds)

        # Change time to bottom of the hour
        ds['time'] = pd.to_datetime(ds['time'].values).floor('h')
        
        monthly_xr_list.append(ds)
    
    
    combined_data = xr.concat(monthly_xr_list, dim='time')
    
    combined_data.to_netcdf(f'../../data/tempo_data/no2_monthly_files/tempo_no2_{month_str}.nc')
    
    
    

/var/tmp/ipykernel_13439/1458978332.py:21: RuntimeWarning: All-NaN slice encountered
  initial_value = np.nanmedian(data_copy)


In [ ]:
day_timestamps

In [ ]:
ds = xr.open_dataset(f'../../data/tempo_data/no2_monthly_files/tempo_no2_{month_str}.nc')
ds

In [ ]:
ds

In [ ]:
# ds = xr.open_dataset(f'../../data/tempo_data/no2_daily_files/tempo_{day_str}.nc', engine='netcdf4')

# ds['time'] = np.array(ds['time'], dtype='datetime64[ns]')
# day_timestamps = np.array(day_timestamps, dtype='datetime64[ns]')
# ds = ds.sel(time=day_timestamps)

# lat_new = np.arange(lat_min+0.005, lat_max, 0.01)
# lon_new = np.arange(lon_min+0.005, lon_max, 0.01)

# # Divide by 1000000000000000
# ds['vertical_column_troposphere'] = ds['vertical_column_troposphere']/1000000000000000

# #fill nas
# for var in list(ds.data_vars.keys()):
    
#     if var == 'main_data_quality_flag':
#         ds['main_data_quality_flag']=ds['main_data_quality_flag'].fillna(2)
#     else:
#         inpainted_results = np.empty_like(ds[f"{var}"])

#         for i in range(len(ds.time)):
#             data = ds.isel(time=i)[f"{var}"]
#             data_arr = np.array(data)
#             filled = inpainting(data_arr)

#             inpainted_results[i, :, :] = filled

#         ds[f"{var}"] = (('time', 'latitude', 'longitude'), inpainted_results)
    
# ds_out = xr.Dataset(
#     {
#         "latitude": (["latitude"], lat_new),
#         "longitude": (["longitude"], lon_new),
#     }
# )

# # Create the regridder object.
# regridder = xe.Regridder(ds, ds_out, "bilinear")

# # Apply the regridding operation.
# ds = regridder(ds)

# # Change time to bottom of the hour
# ds['time'] = pd.to_datetime(ds['time'].values).floor('h')

# monthly_xr_list.append(ds)

In [ ]:
import matplotlib.pyplot as plt
# # Define your latitude and longitude bounds
# lat_min, lat_max = 28.6, 33.4  # Example latitude range
# lon_min, lon_max = -98.9, -88.3  # Example longitude range

# Select one hour of temperature data (e.g., the first timestamp)
hour_index = 0 #Change this to select a different hour if desired
temperature_data = ds_regrid[f"eff_cloud_fraction"].isel(time=hour_index)
# temperature_data = ds['Percent_Tree_Cover']

# Plot the data with switched axes
plt.figure(figsize=(10, 6))

# Plot with latitude on x-axis and longitude on y-axis
temperature_data.T.plot(
    cmap="coolwarm",  # Colormap for temperature visualization
    cbar_kwargs={'label': 'Temperature (K)'}  # Add color bar label
)

# Update axis labels
plt.xlabel("Latitude")
plt.ylabel("Longitude")

plt.show()

In [ ]:
unique_months.astype(str)

In [ ]:
timestamps_dt